# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine

import sglang as sgl
import asyncio

llm = sgl.Engine(model_path="meta-llama/Meta-Llama-3.1-8B-Instruct")

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:00,  5.00it/s]


Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.75it/s]


Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:01<00:00,  1.38it/s]


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.22it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.38it/s]



  0%|          | 0/23 [00:00<?, ?it/s]

...

 83%|████████▎ | 19/23 [00:06<00:01,  3.91it/s]

 87%|████████▋ | 20/23 [00:06<00:00,  3.92it/s]

 91%|█████████▏| 21/23 [00:06<00:00,  3.92it/s]

 96%|█████████▌| 22/23 [00:07<00:00,  3.88it/s]

100%|██████████| 23/23 [00:07<00:00,  3.13it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  [Your Name] and I'm a therapist at [Your Practice/Therapy Center]. Welcome to our center. My specialty is helping individuals, couples, and families work through challenges and find balance in their lives. I'm glad you're taking the first step towards seeking help, and I'm here to support you.
Before we begin, I want to assure you that everything discussed in our sessions is confidential and protected by law. I'll do my best to create a safe, non-judgmental space for you to express yourself freely.

To get started, can you tell me a little bit about what brings you here today?
Prompt: The president of the United States is
Generated text:  the most powerful person in the world. This president is Donald Trump. The president is elected by the American people to make important decisions on their behalf.
President Donald Trump has a very busy schedule. He makes important decisions about the country, meets with foreign leaders, and works with Congre

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing synchronous streaming generation ===")

for prompt in prompts:
    print(f"\nPrompt: {prompt}")
    print("Generated text: ", end="", flush=True)

    for chunk in llm.generate(prompt, sampling_params, stream=True):
        print(chunk["text"], end="", flush=True)
    print()


=== Testing synchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 J

avi

 and

 I

...

.

 Open

CV

's

 open

### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Hello, my name is
Generated text:  René and I am a proud owner of a 1986 Mercedes-Benz 190E.
The 190E is a beautiful and refined car that exudes elegance and sophistication. I am honored to be the owner of this classic Mercedes-Benz model and I am excited to share my experience with you.

I purchased my 190E about a year ago, and since then, I have been thoroughly enjoying driving and maintaining it. The car has been well-maintained and has been kept in great condition, which is a testament to its original owner's dedication to preserving the vehicle.

Over the years, I have been learning more about the car

Prompt: The capital of France is
Generated text:  Paris, located in the north-central region of the country. Paris is known as the "City of Light" (La Ville Lumière) and is famous for its stunning architecture, art museums, fashion, and cuisine. Some of the most famous landmarks in Paris include the Eiffel Tower, the Louvre Museum, Notre Dame Cathedral, and the Champs-Élys

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        generator = await llm.async_generate(prompt, sampling_params, stream=True)
        async for chunk in generator:
            print(chunk["text"], end="", flush=True)
        print()


asyncio.run(main())


=== Testing asynchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 John

,

 and

 I

...

 can

 automate

 repetitive

 tasks

,

In [6]:
llm.shutdown()